In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
import os
import pickle

df_events = pd.read_csv("../databases/events.csv")
df_order_items = pd.read_csv("../databases/order_items.csv")
df_orders = pd.read_csv("../databases/orders.csv")
df_products = pd.read_csv("../databases/products.csv")
df_reviews = pd.read_csv("../databases/reviews.csv")
df_users = pd.read_csv("../databases/users.csv")

ruta_actual = os.getcwd()
mlflow.set_tracking_uri(f"file:///{ruta_actual}/mlruns")
mlflow.set_experiment("Modelos de Recomendación - Comparativa")

<Experiment: artifact_location=('file:///c:\\Users\\fredd\\Desktop\\Soy '
 'Henry\\ProyectoFinal\\PF-Quantum_Insights\\modelos_recomendacion/mlruns/571001947788603457'), creation_time=1769455506279, experiment_id='571001947788603457', last_update_time=1769455506279, lifecycle_stage='active', name='Modelos de Recomendación - Comparativa', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [27]:
def preparar_nlp(df):
    # Solo nos interesa limpiar la columna del nombre
    # No concatenamos ni marca ni categoría para evitar sesgos
    df['perfil_texto'] = df['ProductName'].fillna('').astype(str).str.lower().str.strip()
    
    return df

In [28]:
def recomendar_nlp(product_id, df_sim, top_n=3):
    # Validación básica: si no existe el ID, retorna lista vacía
    if product_id not in df_sim.index:
        return []
    
    # Ordenamos por similitud descendente y eliminamos el propio producto
    similares = df_sim[product_id].sort_values(ascending=False).drop(product_id, errors='ignore')
    
    # Retornamos solo los índices (IDs) de los top_n
    return similares.head(top_n).index.tolist()

In [35]:

df_productos = preparar_nlp(pd.read_csv("../databases/products.csv"))
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_productos['perfil_texto'])
cos_sim = cosine_similarity(tfidf_matrix)
df_sim_nlp = pd.DataFrame(cos_sim, index=df_productos['product_id'], columns=df_productos['product_id'])


recomendaciones = recomendar_nlp("P000001", df_sim_nlp)
print(recomendaciones)

['P000002', 'P000573', 'P000190']


In [30]:
# class ModeloNLP:
#     def __init__(self, df_sim):
#         self.df_sim = df_sim

#     def recomendar(self, product_id, top_n=3):
#         if product_id not in self.df_sim.index:
#             return []

#         similares = (
#             self.df_sim[product_id]
#             .sort_values(ascending=False)
#             .drop(product_id, errors='ignore')
#         )

#         return similares.head(top_n).index.tolist()


In [31]:
# ruta_actual = os.getcwd()

# # Identificar la carpeta correcta donde se guardarán los modelos
# if os.path.exists(os.path.join(ruta_actual, "modelos_entrenados")):
#     ruta_modelos = os.path.join(ruta_actual, "modelos_entrenados")
# else:
#     # Buscar la carpeta subiendo un nivel si no está en el actual
#     ruta_modelos = os.path.join(os.path.dirname(ruta_actual), "modelos_entrenados")

# # Crear la carpeta automáticamente si no existe
# if not os.path.exists(ruta_modelos):
#     os.makedirs(ruta_modelos)

# # Construir la ruta completa con el nombre del archivo final
# archivo_salida = os.path.join(ruta_modelos, "modelo_recomendacion_nlp.pkl")

# modelo_nlp = ModeloNLP(df_sim_nlp)

# archivo_salida = os.path.join(ruta_modelos, "modelo_recomendacion_nlp.pkl")

# with open(archivo_salida, "wb") as f:
#     pickle.dump(modelo_nlp, f)

# print(f"💾 Modelo NLP guardado en: {archivo_salida}")


In [32]:
def evaluar_confianza_cobertura(df_sim, df_todos_productos, k=3, n_muestras=1000):
    ids_disponibles = df_sim.index.tolist()
    total_catalogo = len(df_todos_productos)
    if n_muestras > len(ids_disponibles):
        muestras = ids_disponibles
        print(f"Muestra mayor al catalogo. Evaluando TODO el catálogo ({len(muestras)} items)")
    else:
        muestras = np.random.choice(ids_disponibles, n_muestras, replace=False)
        print(f"Evaluando modelo con una muestra aleatoria de {len(muestras)} productos")

    scores_acumulados = []
    items_recomendados_unicos = set()
    for prod_id in muestras:
        sim_series = df_sim.loc[prod_id]
        top_recs = sim_series.drop(prod_id, errors='ignore').nlargest(k)
        scores_acumulados.extend(top_recs.values)
        items_recomendados_unicos.update(top_recs.index)
        
    # --- CALCULOS FINALES ---
    
    # 1. Confianza (Similitud Promedio)
    confianza_promedio = np.mean(scores_acumulados) if scores_acumulados else 0
    
    # 2. Cobertura
    cobertura = len(items_recomendados_unicos) / total_catalogo
    
    # --- REPORTE ---
    print("\n" + "="*50)
    print(f"Evaluacion de cobertura y confianza (Muestra: {len(muestras)})")
    print("="*50)
    
    # Reporte de Confianza
    print(f"CONFIANZA (Similitud Promedio): {confianza_promedio:.4f}")
    if confianza_promedio > 0.5:
        print("El modelo encuentra similitudes fuertes (textos muy parecidos).")
    else:
        print("Las similitudes son bajas. El modelo podría estar recomendando cosas poco relacionadas.")
        
    print("-" * 50)
    
    # Reporte de Cobertura
    print(f"COBERTURA DE CATALOGO: {cobertura:.2%} ({len(items_recomendados_unicos)}/{total_catalogo} productos)")
    if cobertura < 0.10:
        print("Advertencia, siempre se recomienda los mismos pocos productos.")
    elif cobertura > 0.80:
        print("   ✅ Excelente diversidad. Tu modelo explora casi todo el catálogo.")
    else:
        print("Cobertura moderada.")
        
    print("="*50)
    
    return confianza_promedio, cobertura
confianza, cobertura = evaluar_confianza_cobertura(df_sim_nlp, df_productos, k=3, n_muestras=1000)

Evaluando modelo con una muestra aleatoria de 1000 productos

Evaluacion de cobertura y confianza (Muestra: 1000)
CONFIANZA (Similitud Promedio): 0.5626
El modelo encuentra similitudes fuertes (textos muy parecidos).
--------------------------------------------------
COBERTURA DE CATALOGO: 73.75% (1475/2000 productos)
Cobertura moderada.


In [33]:
def compute_ap(predicciones, objetivos, k):
    """Cálculo de Average Precision (AP) para una consulta"""
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicciones):
        if p in objetivos:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(objetivos), k)

def compute_ndcg(predicciones, objetivos, k):
    """Cálculo de NDCG - qué tan cerca se está de un ranking ideal"""
    dcg = 0.0
    idcg = 0.0
    for i, p in enumerate(predicciones):
        if p in objetivos:
            dcg += 1.0 / np.log2(i + 2)
    num_posibles = min(len(objetivos), k)
    for i in range(num_posibles):
        idcg += 1.0 / np.log2(i + 2)
    
    return dcg / idcg if idcg > 0 else 0.0

def evaluar_nlp_baskets(df_test, df_sim_nlp, k=3):
    ordenes_test = df_test.groupby('order_id')['product_id'].apply(list)
    
    # diccionario de metricas
    metrics = {
        'Precision': [], 'Recall': [], 'F1': [],
        'MRR': [], 'MAP': [], 'NDCG': []
    }

    for items in ordenes_test:
        if len(items) < 2: continue
        for i in range(len(items)):
            semilla = items[i]
            objetivos = set(items[:i] + items[i+1:])
            predicciones = recomendar_nlp(semilla, df_sim_nlp, top_n=k)
            
            if not predicciones: continue

            # Calculos de metricas
            aciertos = len(set(predicciones) & objetivos)
            
            # Precision & Recall
            prec = aciertos / k
            rec = aciertos / len(objetivos)
            metrics['Precision'].append(prec)
            metrics['Recall'].append(rec)
            
            # F1-Score
            if (prec + rec) > 0:
                f1 = 2 * (prec * rec) / (prec + rec)
            else:
                f1 = 0.0
            metrics['F1'].append(f1)
            
            # MRR
            rank_score = 0
            for rank, p_id in enumerate(predicciones, 1):
                if p_id in objetivos:
                    rank_score = 1 / rank
                    break
            metrics['MRR'].append(rank_score)
            
            # MAP & NDCG
            metrics['MAP'].append(compute_ap(predicciones, objetivos, k))
            metrics['NDCG'].append(compute_ndcg(predicciones, objetivos, k))

    # --- REPORTE FINAL ---
    print("\n" + "="*50)
    print(f"📊 REPORTE FINAL NLP (K={k})")
    print("="*50)
    print(f"{'Metrica':<15} | {'Valor':<10}")
    print("-" * 50)
    # Promediamos todos los scores acumulados
    print(f"{'Precision':<15} | {np.mean(metrics['Precision']):.4f}")
    print(f"{'Recall':<15} | {np.mean(metrics['Recall']):.4f}")
    print(f"{'F1-Score':<15} | {np.mean(metrics['F1']):.4f}")
    print("-" * 50)
    print(f"{'MRR':<15} | {np.mean(metrics['MRR']):.4f}")
    print(f"{'MAP':<15} | {np.mean(metrics['MAP']):.4f}")
    print(f"{'NDCG':<15} | {np.mean(metrics['NDCG']):.4f}")
    print("="*50)
    
    name = f"NLP_K{k}"
    with mlflow.start_run(run_name=name):

        # --- Parámetros ---
        mlflow.log_param("modelo", "NLP")
        mlflow.log_param("k", k)

        # --- Métricas ---
        mlflow.log_metric("precision", np.mean(metrics['Precision']))
        mlflow.log_metric("recall", np.mean(metrics['Recall']))
        mlflow.log_metric("f1", np.mean(metrics['F1']))
        mlflow.log_metric("mrr", np.mean(metrics['MRR']))
        mlflow.log_metric("map", np.mean(metrics['MAP']))
        mlflow.log_metric("ndcg", np.mean(metrics['NDCG']))
    return metrics

resultados_nlp = evaluar_nlp_baskets(test_data, df_sim_nlp, k=5)


📊 REPORTE FINAL NLP (K=5)
Metrica         | Valor     
--------------------------------------------------
Precision       | 0.0056
Recall          | 0.0137
F1-Score        | 0.0079
--------------------------------------------------
MRR             | 0.0108
MAP             | 0.0054
NDCG            | 0.0091


mlflow ui --backend-store-uri ./mlruns